[2] 리텐션 분석 : 시간의 흐름에 따라 고객들은 얼마나 남고 이탈했는가?

=> 특정 월 이후 다음 월에 고객이 이탈하는가 남아있는가

In [10]:
import pandas as pd

df = pd.read_csv('data/ecommerce_data_final.csv')

In [11]:
retention_base = df[['CustomerID', 'InvoiceNo', 'date_ymd']].drop_duplicates().reset_index(drop=True)
retention_base

,CustomerID,InvoiceNo,date_ymd
0,17850,536365,2010-12-01
1,17850,536366,2010-12-01
2,13047,536367,2010-12-01
3,13047,536368,2010-12-01
4,13047,536369,2010-12-01
...,...,...,...
18531,13777,581583,2011-12-09
18532,13777,581584,2011-12-09
18533,15804,581585,2011-12-09
18534,13113,581586,2011-12-09


In [12]:
retention_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18536 entries, 0 to 18535
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   CustomerID  18536 non-null  int64 
 1   InvoiceNo   18536 non-null  int64 
 2   date_ymd    18536 non-null  object
dtypes: int64(2), object(1)
memory usage: 434.6+ KB


In [15]:
retention_base['date_ymd'] = pd.to_datetime(retention_base['date_ymd'])

retention_base['date_ym'] = retention_base['date_ymd'].dt.to_period('M')

retention_base.head()

,CustomerID,InvoiceNo,date_ymd,date_ym
0,17850,536365,2010-12-01,2010-12
1,17850,536366,2010-12-01,2010-12
2,13047,536367,2010-12-01,2010-12
3,13047,536368,2010-12-01,2010-12
4,13047,536369,2010-12-01,2010-12


날짜 범위 확인

In [22]:
print(min(retention_base['date_ymd'].unique()))
print(max(retention_base['date_ymd'].unique()))

2010-12-01 00:00:00
2011-12-09 00:00:00


2011년 12월 데이터가 다른 달에 비해 9일에서 끝나므로 자료의 수를 맞추기 위해 2011년 11월까지로 데이터 제한

In [23]:
retention_base = retention_base[retention_base['date_ymd'] <= '2011-11-30'].reset_index(drop = True)
retention_base

,CustomerID,InvoiceNo,date_ymd,date_ym
0,17850,536365,2010-12-01,2010-12
1,17850,536366,2010-12-01,2010-12
2,13047,536367,2010-12-01,2010-12
3,13047,536368,2010-12-01,2010-12
4,13047,536369,2010-12-01,2010-12
...,...,...,...,...
17753,15544,579868,2011-11-30,2011-11
17754,12437,579870,2011-11-30,2011-11
17755,14085,579872,2011-11-30,2011-11
17756,12429,579881,2011-11-30,2011-11


## 리텐션 계산

In [27]:
date_ym_list = sorted(list(retention_base['date_ym'].unique()))
date_ym_list

[Period('2010-12', 'M'),
 Period('2011-01', 'M'),
 Period('2011-02', 'M'),
 Period('2011-03', 'M'),
 Period('2011-04', 'M'),
 Period('2011-05', 'M'),
 Period('2011-06', 'M'),
 Period('2011-07', 'M'),
 Period('2011-08', 'M'),
 Period('2011-09', 'M'),
 Period('2011-10', 'M'),
 Period('2011-11', 'M')]

In [31]:
# t-1 달
period_start = date_ym_list[0]   # 2010-12

# t 달
period_target = date_ym_list[1]  # 2011-01

In [37]:
# t-1 달의 고객 목록 집합
period_start_users = set(retention_base[retention_base['date_ym'] == period_start]['CustomerID'])    # 2010-12

# t 달의 고객 목록 집합
period_target_users = set(retention_base[retention_base['date_ym'] == period_target]['CustomerID'])  # 2011-01

In [ ]:
# set(retention_base[retention_base['date_ym'] == period_target]['CustomerID'])

In [43]:
# 위의 두 집합에서 교집합을 통해 t-1 부터 t 까지 두 달 연속 방문 고객 확인

retained_users = period_start_users.intersection(period_target_users)   # 잔존 고객

# 리텐션 비율 : 잔존 고객 / t-1달 고객
retention_rate = len(retained_users) / len(period_start_users)
round(retention_rate, 2)

0.37

### 반복문을 통해 해당 연월 동안 리텐션 비율 측정

In [47]:
from tqdm import tqdm

In [51]:
retention = pd.DataFrame()
for s in tqdm(date_ym_list):
    for t in date_ym_list:
        period_start = s
        period_target = t

        if period_start <= period_target:
            period_start_users = set(retention_base[retention_base['date_ym'] == period_start]['CustomerID'])
            period_target_users = set(retention_base[retention_base['date_ym'] == period_target]['CustomerID'])

            retained_users = period_start_users.intersection(period_target_users)

            retention_rate = len(retained_users) / len(period_start_users)

            temp = pd.DataFrame({'cohort' : [period_start], 'date_ym' : [period_target], 'retention_rate' : [retention_rate]})

            retention = pd.concat([retention, temp])

100%|██████████| 12/12 [00:00<00:00, 21.92it/s]


In [52]:
retention

,cohort,date_ym,retention_rate
0,2010-12,2010-12,1.000000
0,2010-12,2011-01,0.366102
0,2010-12,2011-02,0.323164
0,2010-12,2011-03,0.384181
0,2010-12,2011-04,0.362712
...,...,...,...
0,2011-09,2011-10,0.386256
0,2011-09,2011-11,0.522117
0,2011-10,2011-10,1.000000
0,2011-10,2011-11,0.456012


In [53]:
# 시작월부터 몇개월 지났는지
retention['cohort_size(month)'] = retention.apply(lambda x : (x['date_ym'] - x['cohort']).n, axis = 1)
retention.head()

,cohort,date_ym,retention_rate,cohort_size(month)
0,2010-12,2010-12,1.000000,0
0,2010-12,2011-01,0.366102,1
0,2010-12,2011-02,0.323164,2
0,2010-12,2011-03,0.384181,3
0,2010-12,2011-04,0.362712,4


In [54]:
retention['cohort'] = retention['cohort'].astype(str)
retention['date_ym'] = retention['date_ym'].astype(str)

retention_final = pd.pivot_table(data = retention , index = 'cohort', columns = 'cohort_size(month)', values = 'retention_rate')
retention_final

cohort_size(month),0,1,2,3,4,5,6,7,8,9,10,11
cohort,,,,,,,,,,,,
2010-12,1.0,0.366102,0.323164,0.384181,0.362712,0.397740,0.362712,0.349153,0.353672,0.395480,0.374011,0.502825
2011-01,1.0,0.353576,0.430499,0.383266,0.457490,0.419703,0.391363,0.384615,0.438596,0.417004,0.514170,NaN
2011-02,1.0,0.382586,0.375989,0.468338,0.428760,0.405013,0.402375,0.440633,0.414248,0.485488,NaN,NaN
2011-03,1.0,0.312115,0.441478,0.401437,0.377823,0.360370,0.421971,0.390144,0.474333,NaN,NaN,NaN
2011-04,1.0,0.429907,0.419393,0.407710,0.387850,0.450935,0.417056,0.491822,NaN,NaN,NaN,NaN
2011-05,1.0,0.388258,0.407197,0.405303,0.442235,0.416667,0.501894,NaN,NaN,NaN,NaN,NaN
2011-06,1.0,0.368315,0.409687,0.467205,0.435923,0.517659,NaN,NaN,NaN,NaN,NaN,NaN
2011-07,1.0,0.408851,0.457323,0.457323,0.521602,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-08,1.0,0.454545,0.449198,0.534759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


시각화

In [56]:
import plotly.express as px

fig = px.imshow(retention_final, text_auto = '.2%', color_continuous_scale = 'Burg')
fig.show()

- 상반기보다 하반기의 1달 뒤 리텐션이 더 높아짐
- 대각선 리텐션이 대체로 높음 -> 2011년 11월 많은 유저가 복귀
- 같은 이유로 2011년 9월에도 많은 유저가 복귀한 것으로 보임

리텐션 커브

In [58]:
retention_curve = retention.groupby('cohort_size(month)')[['retention_rate']].mean().reset_index()
retention_curve

,cohort_size(month),retention_rate
0,0,1.000000
1,1,0.391502
2,2,0.423605
3,3,0.434391
4,4,0.426799
5,5,0.424012
6,6,0.416228
7,7,0.411273
8,8,0.420212
9,9,0.432657


In [59]:
fig = px.line(data_frame = retention_curve, x = 'cohort_size(month)', y = 'retention_rate', title = '리텐션 커브')
fig.update_yaxes(tickformat = '.2%')
fig.show()


리텐션이 보통 cohort_size가 커지면 대체로 retention rate가 내려가는데 반해 해당 이커머스는 리텐션 비율이 유지에서 최근 높아지는 중